<a href="https://colab.research.google.com/github/kvfrans/aitype/blob/main/aitype2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI STANDUP: Tell a story that the AI enjoys!
this code is a slightly modified version of [this](https://github.com/gpt2ent/gpt2colab-js)


**STEPS:**

1. Runtime -> Change runtime type -> Hardware accelerator -> GPU
2. Runtime -> Reset all runtimes
3. Runtime -> Run all


4. The AI will give you a prompt, try to write a scenario that matches!
5. Start typing in the input bar. Finishing a sentence will send it to the AI.
6. The greener your text, the better you are doing.
7. Don't let the bar run out!


In [1]:
#@title Load Stuff {vertical-output: true}
# !pip install transformers

import torch
import torch.nn.functional as F
# from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AutoTokenizer, AutoModelWithLMHead
import google.colab.output
  
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")
model.eval()
model.to('cuda')
torch.set_grad_enabled(False)

# Encode a text inputs
text = "I got a free candy"
import google.colab.output

def ai_evaluate(text, adjs):
    rets = []
    with torch.no_grad():
        full_text = text + ". Overall, this is"
        tokens = [tokenizer.encode(full_text)]
        tokens_tensor = torch.tensor(tokens, dtype=torch.int).to('cuda')
        outputs = model(tokens_tensor)
        all_probs = F.softmax(outputs[0][0][-1], dim=0)
        for a in adjs:
            p = tokenizer.encode(" "+a)[0]
            prob = all_probs[p]
            rets.append(prob.item())
            # print(full_text + tokenizer.decode(p), prob.item())
        # print([tokenizer.decode(x) for x in torch.topk(all_probs, 50)[1]])
    return rets
google.colab.output.register_callback('ai_evaluate', ai_evaluate)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [9]:
#@title The Game {vertical-output: true}
from IPython.display import HTML

html = """
<style>
    .game {
        font-size: 24px;
    }
</style>
<div class="game">
    <div>
    <span id="log_color"></span>
    <span id="log"></span>
    <br>
    <input style="width:80%" class="game" id="user_input" placeholder="Type here!"></input>
    <div style="width:80%; background-color: rgb(100, 200, 100);" class="game" id="typing_bar">.</div>
    </div>
</game>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
<script type="text/Javascript">
console.log("hey")
var time_type = 0.5;
var started = false;
var num_sents_left = 5;
var all_words = [
    ["good", "bad"],
    ["funny", "boring"],
    ["stupid", "smart"],
    ["weird", "normal"],
    ["green", "red"],
    ["fast", "slow"],
    ["depressing", "uplifting"],
]
var words = ["good", "bad"];
var kernel = google.colab.kernel;
inp = document.getElementById('user_input');
log = document.getElementById('log');

words = all_words[Math.floor(Math.random() * all_words.length)]
log_color.innerHTML += "<br><b> Write a "+words[0].toUpperCase()+" (not "+words[1].toUpperCase()+") story.</b><br>"


inp.addEventListener("keyup", function(event) {
    time_type += 0.05;
    started = true;
    if (event.keyCode == 32) {
        var text = inp.value;
        if (text.includes(".")) {
            inp.value = "";
            log.innerHTML += text
            var process_text = log.innerHTML;
            var resultPromise = kernel.invokeFunction("ai_evaluate", [process_text, words]); // developer, look here
            log.innerHTML = "";
            resultPromise.then( function(value) {
                vals = JSON.parse(value.data['text/plain']);
                t = vals[0] / (vals[0] + vals[1])
                r = Math.floor(255-t*255)
                g = Math.floor(t*255)
                log_color.innerHTML += " <span style='color:rgb("+r+","+g+",0)'>" + process_text + "</span>";
                num_sents_left -= 1;
                console.log(num_sents_left);
                if (num_sents_left <= 0) {
                    num_sents_left = 5;
                    words = all_words[Math.floor(Math.random() * all_words.length)]
                    if (Math.random() < 0.5) {
                        tmp = words[0]
                        words[0] = words[1]
                        words[1] = tmp
                    }
                    console.log("here")
                    log_color.innerHTML += "<br><b> Write a "+words[0].toUpperCase()+" (not "+words[1].toUpperCase()+") story.</b><br>"
                }
            });
        }
    }
});

typing = document.getElementById('typing_bar');
function tick() {
    if(started) {
        time_type -= 0.01;
        typing.style.width = (80*time_type)+"%";
        time_type = Math.min(Math.max(time_type, 0), 1);
    }
}

setInterval(tick,100);

</script>
"""

HTML(html)